# Analizar Resultados de Prueba A/B

(Inspirado en un proyecto público de Udacity).

Este proyecto asegurará que has dominado los temas cubiertos en las lecciones de estadística. Hemos organizado el cuaderno actual en las siguientes secciones:

- [Introducción](#intro)
- [Parte I - Probabilidad](#probability)
- [Parte II - Prueba A/B](#ab_test)



Las tareas específicas de programación están marcadas con una etiqueta **ToDo**.

<a id='intro'></a>
## Introducción

Las pruebas A/B son muy comúnmente realizadas por analistas de datos y científicos de datos. Para este proyecto, trabajarás para entender los resultados de una prueba A/B realizada por un sitio web de comercio electrónico. Tu objetivo es trabajar a través de este cuaderno para ayudar a la empresa a entender si deben:
- Implementar la nueva página web,
- Mantener la antigua página web, o
- Quizás ejecutar el experimento por más tiempo para tomar su decisión.

<a id='probability'></a>
## Parte I - Probabilidad

Para empezar, vamos a importar nuestras bibliotecas.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#Fijamos la semilla para asegurarnos de obtener las mismas respuestas
random.seed(42)

### ToDo 1.1
Ahora, lee los datos de `ab_data.csv`. Almacénalos en `df`. A continuación, se describe la data, que consta de un total de 5 columnas:

<center>

|Columnas de datos|Propósito|Valores válidos|
| ------------- |:-------------| -----:|
|user_id|ID único|Valores Int64|
|timestamp|Marca de tiempo cuando el usuario visitó la página web|-|
|group|En el actual experimento A/B, los usuarios se categorizan en dos grupos amplios. <br>Se espera que a los usuarios del grupo `control` se les muestre la `old_page`; y a los usuarios del grupo `treatment` se les presenta la `new_page`. <br>Sin embargo, **algunas filas inexactas** están presentes en los datos iniciales, como un usuario del grupo `control` que se empareja con una `new_page`. |`['control', 'treatment']`|
|landing_page|Denota si el usuario visitó la antigua o la nueva página web.|`['old_page', 'new_page']`|
|converted|Denota si el usuario decidió pagar por el producto de la empresa. Aquí, `1` significa que sí, el usuario compró el producto.|`[0, 1]`|
</center>




>

**a.** Lee el conjunto de datos del archivo `ab_data.csv` y echa un vistazo a las primeras filas aquí:

In [8]:
df = pd.read_csv('ab_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


**b.** Usa la celda de abajo para encontrar el número de filas en el conjunto de datos.


In [70]:
df.shape[0]

294478


**c.** El número de usuarios únicos en el conjunto de datos.


In [4]:
nums_unique_users = df['user_id'].nunique()
nums_unique_users

290584


**d.** La proporción de usuarios convertidos.


In [72]:
proporcion_convertidos = round(len(df[df['converted'] == 1]) / nums_unique_users, 3)
porcentaje_convertidos = round(proporcion_convertidos * 100, 3)
print(f'Proporción: {proporcion_convertidos}\nPorcentaje: {porcentaje_convertidos}%')

Proporción: 0.121
Porcentaje: 12.1%



**e.** El número de veces en que el "group" es `treatment` pero "landing_page" no es una `new_page`.


In [6]:
treatment_newpage = df[(df['group'] == 'treatment') & (df['landing_page'] != 'new_page')]
treatment_newpage.shape[0]

1965


**f.** ¿Alguna de las filas tiene valores faltantes?

In [21]:
df.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

### ToDo 1.2
En una fila particular, las columnas **group** y **landing_page** deberían tener uno de los siguientes valores aceptables:

|user_id| timestamp|group|landing_page|converted|
|---|---|---|---|---|
|XXXX|XXXX|`control`| `old_page`|X |
|XXXX|XXXX|`treatment`|`new_page`|X |

Esto significa que los usuarios del grupo `control` deben coincidir con `old_page`; y los usuarios del grupo `treatment` deben coincidir con `new_page`.

Sin embargo, para las filas en las que `treatment` no coincide con `new_page` o `control` no coincide con `old_page`, no podemos estar seguros de si tales filas recibieron realmente la nueva o la antigua página web.

¿cómo debemos manejar las filas donde las columnas de group y landing_page no coinciden?

**a.** usa la respuesta para crear un nuevo conjunto de datos. Almacena tu nuevo dataframe en **df2**.

In [39]:
# Elimina las filas inexactas y guarda el resultado en un nuevo dataframe df2
df2 = df.drop(df[(df['group'] == 'control') & (df['landing_page'] == 'new_page')].index)
df2.drop(df[(df['group'] == 'treatment') & (df['landing_page'] == 'old_page')].index, inplace = True)
# df2

In [40]:
# Verifica nuevamente que todas las filas incorrectas fueron eliminadas de df2 -
# La salida de la siguiente declaración debería ser 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

### ToDo 1.3  


**a.** ¿Cuántos **user_id** únicos hay en **df2**?

In [41]:
nums_unique_users_df2 = df2.nunique().user_id
nums_unique_users_df2

290584

**b.** Hay un **user_id** repetido en **df2**. ¿Cuál es?

In [130]:
user_id_repetido = df2[df2['user_id'].duplicated() == True]
id_repetido = user_id_repetido['user_id'].values[0]
id_repetido

773192

**c.** ¿Muestra las filas para el **user_id** duplicado?

In [131]:
df2[df2['user_id'] == id_repetido]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


**d.** Elimina **una** de las filas con un **user_id** duplicado, del dataframe **df2**.

In [132]:
# Elimina una de las filas con un user_id duplicado..
# Pista: El método dataframe.drop_duplicates() puede que no funcione en este caso porque las filas con user_id duplicado no son totalmente idénticas.
df2.drop(df2[df2['user_id'] == id_repetido].index[0], inplace = True)
# Comprueba de nuevo si la fila con un user_id duplicado ha sido eliminada o no.
df2.shape

(290584, 5)

### ToDo 1.4  

**a.** ¿Cuál es la probabilidad de que un individuo se convierta independientemente de la página que reciba?<br><br>

>**Consejo**: La probabilidad que calcularás representa la tasa de éxito de "conversión" general en la población y puedes llamarla $p_{population}$.

In [135]:
total_population = df2.shape[0]
total_convertion = df2[df2['converted'] == 1].shape[0]
p_population = round(total_convertion / total_population, 3)
p_population

0.12

**b.** Dado que un individuo estaba en el grupo `control`, ¿cuál es la probabilidad de que se haya convertido?

In [141]:
total_control_population = df2[df2['group'] == 'control'].shape[0]
total_control_population_converted = df2[(df2['group'] == 'control') & (df2['converted'] == 1)].shape[0]
p_population_control = round(total_control_population_converted / total_control_population, 3)
p_population_control

0.12

**c.** Dado que un individuo estaba en el grupo `treatment`, ¿cuál es la probabilidad de que se haya convertido?

In [ ]:
juancarmonag@laguna-ai.com

>**Consejo**: Las probabilidades que has calculado en los puntos (b). y (c). arriba también pueden ser tratadas como tasa de conversión.
Calcula la diferencia real (`obs_diff`) entre las tasas de conversión para los dos grupos. Lo necesitarás más tarde.

In [ ]:
# Calcula la diferencia real (obs_diff) entre las tasas de conversión para los dos grupos.

**d.** ¿Cuál es la probabilidad de que un individuo reciba la nueva página?



**e.** Considera tus resultados de las partes (a) a (d) anteriores, y explica a continuación si los nuevos usuarios del grupo `treatment` conducen a más conversiones.


>**Your answer goes here.**


<a id='ab_test'></a>
## Parte II - Prueba A/B

Dado que cada evento está asociado con una marca de tiempo, podrías realizar una prueba de hipótesis continuamente mientras observes los eventos.

Sin embargo, entonces las preguntas difíciles serían:
- ¿Debes parar tan pronto como una página sea considerada significativamente mejor que otra o necesita suceder de manera consistente durante un cierto período de tiempo?
- ¿Cuánto tiempo debes ejecutar para tomar una decisión de que ninguna página es mejor que la otra?

Estas preguntas son las partes difíciles asociadas con las pruebas A/B en general.


### ToDo 2.1
Por ahora, considera que necesitas tomar la decisión solo basándote en todos los datos proporcionados.

> Recuerda que acabas de calcular que la probabilidad (o tasa) de "conversión" para la página antigua es *ligeramente* mayor que la de la nueva página (ToDo 1.4.c).

Si quieres asumir que la página antigua es mejor a menos que la nueva página demuestre ser definitivamente mejor con una tasa de error de Tipo I del 5%, ¿cuáles deberían ser tus hipótesis nula y alternativa (**$H_0$** y **$H_1$**)?

Puedes expresar tu hipótesis en palabras o en términos de **$p_{old}$** y **$p_{new}$**, que son la probabilidad (o tasa) de "conversión" para las páginas antigua y nueva respectivamente.

>**Put your answer here.**

### ToDo 2.2 - Prueba de la hipótesis nula $H_0$
Bajo la hipótesis nula $H_0$, asumimos que $p_{new}$ y $p_{old}$ son iguales. Además, asumimos que $p_{new}$ y $p_{old}$ son ambos iguales a la tasa de éxito de **conversión** en los datos de `df2`, independientemente de la página. Entonces, nuestra suposición es: <br><br>
<center>
$p_{new}$ = $p_{old}$ = $p_{population}$
</center>

En esta sección, tú:

- Simularás (bootstrap) un conjunto de datos de muestra para ambos grupos, y calcularás la probabilidad de "conversión" $p$ para esas muestras.


- Utilizarás un tamaño de muestra para cada grupo igual a los de los datos `df2`.


- Calcularás la diferencia en la probabilidad de "conversión" para las dos muestras anteriores.


- Realizarás la distribución de muestreo para la "diferencia en la probabilidad de conversión" entre las dos muestras simuladas en 10,000 iteraciones; y calcularás una estimación.



Usa las celdas de abajo para proporcionar las partes necesarias de esta simulación.

**a.** ¿Cuál es la **tasa de conversión** para $p_{new}$ bajo la hipótesis nula?




**b.** ¿Cuál es la **tasa de conversión** para $p_{old}$ bajo la hipótesis nula?



**c.** ¿Cuál es $n_{new}$, el número de individuos en el grupo de tratamiento? <br><br>
*Sugerencia*: A los usuarios del grupo de tratamiento se les muestra la nueva página.



**d.** ¿Cuál es $n_{old}$, el número de individuos en el grupo de control?



**e. Simulación de la Muestra para el Grupo `treatment`**<br>
Simula $n_{new}$ transacciones con una tasa de conversión de $p_{new}$ bajo la hipótesis nula.  <br><br>
*Sugerencia*: Usa el método `numpy.random.choice()` para generar aleatoriamente $n_{new}$ valores. <br>
Guarda estos $n_{new}$ 1's y 0's en el array `new_page_converted` de numpy.



In [ ]:
# Simulate a Sample for the treatment Group


**f. Simulación de la Muestra para el Grupo `control`** <br>
Simula $n_{old}$ transacciones con una tasa de conversión de $p_{old}$ bajo la hipótesis nula. <br> Guarda estos $n_{old}$ 1's y 0's en el array `old_page_converted` de numpy.

In [ ]:
# Simulate a Sample for the control Group

**g.** Encuentra la diferencia en la probabilidad de "conversión" $(p{'}_{new}$ - $p{'}_{old})$ para tus muestras simuladas de las partes (e) y (f) anteriores.




**h. Distribución de muestreo** <br>
Recrea `new_page_converted` y `old_page_converted` y encuentra el valor de $(p{'}_{new}$ - $p{'}_{old})$ 10,000 veces usando el mismo proceso de simulación que utilizaste en las partes (a) a (g) anteriores.

<br>
Guarda todos los valores $(p{'}_{new}$ - $p{'}_{old})$ en un array de NumPy llamado `p_diffs`.


In [ ]:
# Sampling distribution
p_diffs = []


**i. Histograma**<br>
Haz un histograma de los **p_diffs**. ¿Este gráfico se parece a lo que esperabas? Usa el problema correspondiente en el aula para asegurarte de que comprendes completamente lo que se calculó aquí.<br><br>

Además, usa el método `plt.axvline()` para marcar la diferencia real observada en los datos de `df2` (recuerda `obs_diff`), en el gráfico.

>**Consejo**: Muestra el título, la etiqueta del eje x y la etiqueta del eje y en el gráfico.



**j.** ¿Qué proporción de los **p_diffs** es mayor que la diferencia real observada en los datos de `df2`?

**k.** Lo que acabas de calcular en la parte **j** se llama valor p en estudios científicos. Este valor representa la probabilidad de obtener los resultados observados (o más extremos) si la hipótesis nula es cierta. En términos de si hay una diferencia entre las nuevas y viejas páginas, si el valor p es menor que el umbral de error de Tipo I (0.05 en este caso), entonces rechazamos la hipótesis nula. En este caso, el valor p es más grande que 0.05, por lo tanto, no rechazamos la hipótesis nula. Esto significa que no hay suficiente evidencia para concluir que la nueva página lleva a más conversiones que la vieja página.


>**Put your answer here.**


**l. Usando Métodos Incorporados para la Prueba de Hipótesis**<br>
También podríamos usar incorporaciones para lograr resultados similares. Aunque el uso de las incorporaciones podría ser más fácil de codificar, las secciones anteriores son un recorrido por las ideas que son fundamentales para pensar correctamente sobre la significación estadística.

Rellena las afirmaciones siguientes para calcular:
- `convert_old`: número de conversiones con la vieja página
- `convert_new`: número de conversiones con la nueva página
- `n_old`: número de individuos a los que se mostró la vieja página
- `n_new`: número de individuos a los que se mostró la nueva página


In [ ]:
import statsmodels.api as sm

# number of conversions with the old_page
convert_old =

# number of conversions with the new_page
convert_new =

# number of individuals who were shown the old_page
n_old =

# number of individuals who received new_page
n_new =

**m.** Ahora usa `sm.stats.proportions_ztest()` para calcular tu estadística de prueba y el valor p. [Aquí](https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html) hay un enlace útil para usar la función incorporada.

La sintaxis es:
```bash
proportions_ztest(count_array, nobs_array, alternative='larger')
```
donde,
- `count_array` = representa el número de "convertidos" para cada grupo
- `nobs_array` = representa el número total de observaciones (filas) en cada grupo
- `alternative` = elige uno de los valores de `[‘two-sided’, ‘smaller’, ‘larger’]` dependiendo de si es de dos colas, de cola izquierda, o de cola derecha respectivamente.

>**Sugerencia**: <br>
Es de dos colas si definiste $H_1$ como $(p_{new} = p_{old})$. <br>
Es de cola izquierda si definiste $H_1$ como $(p_{new} < p_{old})$. <br>
Es de cola derecha si definiste $H_1$ como $(p_{new} > p_{old})$.

La función incorporada anterior devolverá el z_score y el valor p.

---
### Acerca de la prueba z de dos muestras
Recuerda que has trazado una distribución `p_diffs` que representa la
diferencia en la probabilidad "convertida" $(p{'}_{new}-p{'}_{old})$ para tus dos muestras simuladas 10,000 veces.

Otra forma de comparar la media de dos distribuciones independientes y normales es una **prueba z de dos muestras**. Puedes realizar la prueba Z para calcular el Z_score, como se muestra en la ecuación a continuación:

$$
Z_{score} = \frac{ (p{'}_{new}-p{'}_{old}) - (p_{new}  -  p_{old})}{ \sqrt{ \frac{\sigma^{2}_{new} }{n_{new}} + \frac{\sigma^{2}_{old} }{n_{old}}  } }
$$

donde,
- $p{'}$ es la tasa de éxito "convertida" en la muestra
- $p_{new}$ y $p_{old}$ son la tasa de éxito "convertida" para los dos grupos en la población.
- $\sigma_{new}$ y $\sigma_{new}$ son la desviación estándar para los dos grupos en la población.
- $n_{new}$ y $n_{old}$ representan el tamaño de los dos grupos o muestras (es el mismo en nuestro caso)

>La prueba Z se realiza cuando el tamaño de la muestra es grande y se conoce la varianza de la población. El puntaje z representa la distancia entre las dos tasas de éxito "convertidas" en términos del error estándar.

El siguiente paso es tomar una decisión para rechazar o no rechazar la hipótesis nula basándose en la comparación de estos dos valores:
- $Z_{score}$
- $Z_{\alpha}$ o $Z_{0.05}$, también conocido como valor crítico en el intervalo de confianza del 95%.  $Z_{0.05}$ es 1.645 para pruebas de una sola cola, y 1.960 para prueba de dos colas. Puedes determinar el $Z_{\alpha}$ de la tabla z manualmente.

Decide si tu hipótesis es de dos colas, de cola izquierda, o de cola derecha. En consecuencia, rechaza O no rechaza la hipótesis nula basándose en la comparación entre $Z_{score}$ y $Z_{\alpha}$. Determinamos si el $Z_{score}$ está en la "región de rechazo" en la distribución. En otras palabras, una "región de rechazo" es un intervalo donde se rechaza la hipótesis nula si y solo si el $Z_{score}$ se encuentra en esa región.

>Sugerencia:<br>
Para una prueba de cola derecha, rechaza la hipótesis nula si $Z_{score}$ > $Z_{\alpha}$. <br>
Para una prueba de cola izquierda, rechaza la hipótesis nula si $Z_{score}$ < $Z_{\alpha}$.

Referencia:
- Ejemplo 9.1.2 en esta [página](https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/09%3A_Two-Sample_Problems/9.01%3A_Comparison_of_Two_Population_Means-_Large_Independent_Samples), cortesía de www.stats.libretexts.org

---

>**Sugerencia**: No tienes que profundizar en la prueba z para este ejercicio. **Intenta tener una visión general de lo que significa el puntaje z en general.**

In [ ]:
import statsmodels.api as sm
# ToDo: Complete the sm.stats.proportions_ztest() method arguments
z_score, p_value = sm.stats.proportions_ztest(.........)
print(z_score, p_value)

**n.** ¿Qué significan el puntaje z y el valor p que calculaste en la pregunta anterior para las tasas de conversión de las páginas viejas y nuevas? ¿Coinciden con los hallazgos en las partes **j.** y **k.**?<br><br>

>**Sugerencia**: Observa si el valor p es similar al que se calculó anteriormente. En consecuencia, ¿puedes rechazar/no rechazar la hipótesis nula? Es importante interpretar correctamente la estadística de prueba y el valor p.

>**Put your answer here.**